In [10]:
import pandas as pd
import json
from pymongo import MongoClient, ASCENDING, DESCENDING
import mysql.connector
import time
from sqlalchemy import create_engine,text
from datetime import date
import random
from datetime import datetime
import pprint

In [11]:
df_answers = pd.read_csv("dataset/Answers.csv", encoding="ISO-8859-1").dropna()
df_questions = pd.read_csv("dataset/Questions.csv", encoding="ISO-8859-1").dropna()
df_taglink = pd.read_csv("dataset/Tags.csv", encoding="ISO-8859-1").dropna()
df_tags = df_taglink.drop_duplicates(subset=['Tag'], keep='first').drop(columns=['Id'])
df_tags.head()

,Tag
0,python
1,osx
2,fonts
3,photoshop
5,windows


In [12]:
print(f'df_questions')
print({df_questions.info()})
print("\n------------------------------------------\n")
print(f'df_answers')
print(f'{df_answers.info()}')
print("\n------------------------------------------\n")
print(f'df_tags')
print(f'{df_tags.info()}')
print("\n------------------------------------------\n")
print(f'df_taglink')
print(f'{df_taglink.info()}')

df_questions
<class 'pandas.core.frame.DataFrame'>
Index: 601070 entries, 0 to 607281
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            601070 non-null  int64  
 1   OwnerUserId   601070 non-null  float64
 2   CreationDate  601070 non-null  object 
 3   Score         601070 non-null  int64  
 4   Title         601070 non-null  object 
 5   Body          601070 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 32.1+ MB
{None}

------------------------------------------

df_answers
<class 'pandas.core.frame.DataFrame'>
Index: 981755 entries, 0 to 987121
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Id            981755 non-null  int64  
 1   OwnerUserId   981755 non-null  float64
 2   CreationDate  981755 non-null  object 
 3   ParentId      981755 non-null  int64  
 4   Score         981755 non-null 

In [13]:
df_questions['OwnerUserId'] = df_questions['OwnerUserId'].apply(int)
df_questions['Title'] = df_questions['Title'].astype('string')
df_questions['Body'] = df_questions['Body'].astype('string')
df_questions['CreationDate'] = pd.to_datetime(df_questions['CreationDate']).dt.strftime('%Y-%m-%d %H:%M:%S')

df_answers['OwnerUserId'] = df_answers['OwnerUserId'].apply(int)
df_answers['Body'] = df_answers['Body'].astype('string')
df_answers['CreationDate'] = pd.to_datetime(df_answers['CreationDate']).dt.strftime('%Y-%m-%d %H:%M:%S')

In [14]:
print("Tables before filtering:")
print(df_questions.shape)
print(df_answers.shape)
print(df_tags.shape)
print(df_taglink.shape)

df_questions['CreationDate'] = pd.to_datetime(df_questions['CreationDate'])
df_answers['CreationDate'] = pd.to_datetime(df_answers['CreationDate'])

# Filter the data for the years from 2011 to 2016
df_questions = df_questions[df_questions['CreationDate'].dt.year.isin(range(2011, 2016))]
df_answers = df_answers[df_answers['CreationDate'].dt.year.isin(range(2011, 2016))]

df_taglink = df_taglink[df_taglink['Id'].isin(df_questions['Id'])]

print("\nTables after filtering:")
print(df_questions.shape)
print(df_answers.shape)
print(df_tags.shape)
print(df_taglink.shape)

Tables before filtering:
(601070, 6)
(981755, 6)
(16895, 1)
(1884635, 2)

Tables after filtering:
(431737, 6)
(705766, 6)
(16895, 1)
(1334927, 2)


In [15]:
#remove orphan answers
orphans = df_answers[~df_answers['ParentId'].isin(df_questions['Id'])]['ParentId']
print(f"Orphan answers: {len(orphans)}")

df_answers = df_answers[~df_answers['ParentId'].isin(orphans)]
print(f"Orphan answers removed: {len(orphans)}")

Orphan answers: 26536
Orphan answers removed: 26536


In [7]:
# Connect to MySQL

database_name = 'stackoverflow'
# Create Connection
connection = mysql.connector.connect(
  host="localhost",
  user="root", 
  password = '123456789'
)

cursor = connection.cursor()
cursor.execute(f"DROP DATABASE IF EXISTS {database_name}")
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
cursor.execute(f"USE {database_name}")


cursor.execute("DROP TABLE IF EXISTS taglink")
cursor.execute("DROP TABLE IF EXISTS tags")
cursor.execute("DROP TABLE IF EXISTS answers")
cursor.execute("DROP TABLE IF EXISTS questions")

# Creating Table
cursor.execute("""CREATE TABLE questions (
               Id INT AUTO_INCREMENT PRIMARY KEY, 
               OwnerUserId INT NOT NULL, 
               CreationDate DATETIME NOT NULL, 
               Score INT DEFAULT 0, 
               Title VARCHAR(255) NOT NULL, 
               Body TEXT NOT NULL)""")
cursor.execute("""CREATE TABLE answers (
               Id INT AUTO_INCREMENT PRIMARY KEY,
               OwnerUserId INT NOT NULL, 
               CreationDate DATETIME NOT NULL,
               ParentId INT NOT NULL, 
               Score INT DEFAULT 0, 
               Body TEXT NOT NULL,
               FOREIGN KEY (ParentId) REFERENCES questions(Id))""")
cursor.execute("CREATE TABLE tags (Tag VARCHAR(255) PRIMARY KEY)")
cursor.execute("""CREATE TABLE taglink (
               Id INT NOT NULL, 
               Tag VARCHAR(255) NOT NULL,
               FOREIGN KEY (Id) REFERENCES questions(Id),
               FOREIGN KEY (Tag) REFERENCES tags(Tag))""")

connection.commit()

# Check if Table Exists
cursor.execute("SHOW TABLES")
for x in cursor:
    print(x)


def insertData(insert_query, data, table_name, batch_size=1000):
    total_rows_inserted = 0
    for i in range(0, len(data), batch_size):
        batch = data[i:i + batch_size]
        cursor.executemany(insert_query, batch)
        connection.commit()
        total_rows_inserted += cursor.rowcount
    print(f"Inserted {total_rows_inserted} rows into MySQL, in table {table_name}.")

# Insert data
insert_questions = """
INSERT INTO questions (Id, OwnerUserId, CreationDate, Score, Title, Body)
VALUES (%s, %s, %s, %s, %s, %s);
"""
data_to_insert = [(item["Id"], item["OwnerUserId"],item["CreationDate"], item["Score"], item["Title"], item["Body"]) for item in df_questions.to_dict(orient="records")]
insertData(insert_questions, data_to_insert, "Questions")

insert_answers = """
INSERT INTO answers (Id, OwnerUserId, CreationDate, ParentId, Score, Body)
VALUES (%s, %s, %s, %s, %s, %s);
"""
data_to_insert = [(item["Id"], item["OwnerUserId"],item["CreationDate"], item["ParentId"], item["Score"], item["Body"]) for item in df_answers.to_dict(orient="records")]
insertData(insert_answers, data_to_insert, "Answers")

insert_tags = """
INSERT INTO tags (Tag)
VALUES (%s);
"""
data_to_insert = [(item["Tag"],) for item in df_tags.to_dict(orient="records")]
insertData(insert_tags, data_to_insert, "Tags")

insert_taglink = """
INSERT INTO taglink (Id, Tag)
VALUES (%s, %s);
"""
data_to_insert = [(item["Id"], item["Tag"]) for item in df_taglink.to_dict(orient="records")]
insertData(insert_taglink, data_to_insert, "TagLink")

('answers',)
('questions',)
('taglink',)
('tags',)
Inserted 431737 rows into MySQL, in table Questions.
Inserted 679230 rows into MySQL, in table Answers.
Inserted 16895 rows into MySQL, in table Tags.
Inserted 1334927 rows into MySQL, in table TagLink.


In [8]:
#Connect to MySQL with slqAlchemy
# username = 'root'
# password = 'Mysql4ever2021'
# host = 'localhost' 
# port = '3306' 
# database_name = 'stackoverflow'
# questions_table = 'questions'
# answers_table = 'answers'
# tags_table = 'tags'
# taglink_table = 'taglink'

# Create the connection URL for SQL Alchemy
# db_url = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}'
# engine = create_engine(db_url)
# with engine.connect() as connection:
#     connection.execute(text(f"DROP DATABASE IF EXISTS {database_name}"))
#     connection.execute(text(f"CREATE DATABASE {database_name}"))
# db_url = f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database_name}'
# engine = create_engine(db_url)

# df_questions.to_sql(questions_table, con=engine, if_exists='replace', chunksize= 1000, index=False)
# df_answers.to_sql(answers_table, con=engine, if_exists='replace', chunksize= 1000, index=False)
# df_tags.to_sql(tags_table, con=engine, if_exists='replace', chunksize= 1000, index=False)
# df_taglink.to_sql(taglink_table, con=engine, if_exists='replace', chunksize= 1000, index=False)

# connection = engine.connect()
# connection.execute(text("ALTER TABLE questions MODIFY id INT AUTO_INCREMENT PRIMARY KEY;"))
# connection.execute(text("ALTER TABLE answers MODIFY id INT AUTO_INCREMENT PRIMARY KEY;"))
# connection.execute(text("ALTER TABLE tags ADD PRIMARY KEY (Tag(255));"))


In [ ]:
df_questions['answers'] = df_questions['Id'].apply(lambda x: list(df_answers[df_answers['ParentId'] == x]['Id']))
df_questions.head()


In [ ]:
df_questions['tags'] = df_questions['Id'].apply(lambda x: list(df_taglink[df_taglink['Id'] == x]['Tag']))
df_questions.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body,answers,tags
40692,4572755,105066,2011-01-01 00:03:27,9,email client for app engine,<p>GAE supports both incoming and outgoing ema...,"[4771671, 4789266, 5937135]","[python, google-app-engine, email, webmail]"
40693,4572859,441923,2011-01-01 00:58:48,13,Django: How to check if the user left all fiel...,<p>I know <code>is_valid()</code> on a bounded...,"[4572863, 4572907, 4574902, 11334264]","[python, django, django-forms]"
40694,4572880,119215,2011-01-01 01:17:31,3,Strange PYTHONPATH problem,<p>I recently updated my python installation t...,[4573048],"[python, import, importerror]"
40695,4572943,234932,2011-01-01 01:55:07,3,Start a wsgi app from within virtualenv as a L...,<p>I'm currently developing a bottle app withi...,[4632896],"[python, linux, wsgi, bottle]"
40696,4572984,559589,2011-01-01 02:19:28,0,Django Keyword Search Not Working,<p>I bought this book and Im busy working thro...,[4573170],"[python, django]"


In [ ]:
#Convert CreationDate from string to datetime
df_questions['CreationDate'] = pd.to_datetime(df_questions['CreationDate'])
df_answers['CreationDate'] = pd.to_datetime(df_answers['CreationDate'])

In [ ]:
# Setting MongoDB database

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017")

# Drop and create database
client.drop_database("stackoverflow")
db = client['stackoverflow']

# Drop and create collections
db['Questions'].drop()
db['Answers'].drop()
db['Tags'].drop()
db['TagLink'].drop()
collection_questions = db['Questions']
collection_answers = db['Answers']
collection_tags = db['Tags']
collection_taglink = db['TagLink']

# Function to insert data into MongoDB
def insert_into_mongo(df, collection):
    initial_count = collection.count_documents({})
    print(f"# documents: {initial_count}") 
    try:
        data_dict = df.to_dict(orient="records")
        collection.insert_many(data_dict)
        valid_count = collection.count_documents({}) - initial_count
        print(f"{valid_count} documents inserted.")
    except Exception as e:
        attempted_count = len(data_dict)
        valid_count = collection.count_documents({}) - initial_count
        print(f"{valid_count} documents inserted")
        print(f"{attempted_count - valid_count} documents failed to insert")

insert_into_mongo(df_questions, collection_questions)
insert_into_mongo(df_answers, collection_answers)
insert_into_mongo(df_taglink, collection_taglink)
insert_into_mongo(df_tags, collection_tags)

# documents: 0
431737 documents inserted.
# documents: 0
679230 documents inserted.
# documents: 0
1334927 documents inserted.
# documents: 0
16895 documents inserted.


In [ ]:
queries = [
    {"description": "Retrieve title of all questions that have a score greater than 300", "query": ""},
    
    {"description": "Retrieve all answers from 2014", "query": ""},
    
    {"description": "Delete tag python for the oldest question from 2015 that has at least 2 tags", "query": ""},

    {"description": "Insert a test answer for the oldest question from 2015 containing the word pdf in the title and that has at least 4 answers", "query": ""}

]

In [ ]:
from bson.objectid import ObjectId
mongo_times = []
for q in queries:
    start_time = time.time()
    
    if "score greater" in q["description"]:
        query = {'Score': {'$gt': 300}}
        result = collection_questions.find(query, {'Title': 1, '_id': 0})
        count = collection_questions.count_documents(query)

    elif "from 2014" in q["description"]:
        query = {
            'CreationDate': {
                '$gte': datetime(2014, 1, 1),
                '$lte': datetime(2014, 12, 31)
            }
        }
        result = collection_answers.find(query)
        count = collection_answers.count_documents(query)

    elif "2 tags" in q["description"]:
        query = {
            'CreationDate': {
                '$gte': datetime(2015, 1, 1),
                '$lte': datetime(2015, 12, 31)
            },
            '$expr': {'$gte': [{'$size': '$tags'}, 2]}
        }
        result = collection_questions.find_one(query, sort=[('CreationDate', 1)])
        if result:
            collection_questions.update_one(
                {'_id': result['_id']},
                {'$pull': {'tags': 'python'}}
            )
            collection_taglink.delete_one({'Id': result['Id']})
            print(f"The deleted tag is: python, the question ID is: {result['_id']}")

        count = collection_questions.count_documents(query)
    else:
        query = {'CreationDate': {
                '$gte': datetime(2015, 1, 1),
                '$lte': datetime(2015, 12, 31)
            }, 
            'Title': {'$regex' : '.*pdf.*'}, 
            '$expr': {'$gte': [{'$size': '$answers'}, 4]}
            }
        result = collection_questions.find_one(query, sort=[('CreationDate', 1)])
        if result:
            last_id = collection_answers.find_one(sort=[('Id', -1)])['Id']
            parent_id = result['Id']
            parent_object_id = result['_id']
            new_answer = {
                'AnswerText': 'This is the new answer for this question.', 
                'CreationDate': datetime.now(),
                'OwnerUserId': random.randint(1, 1000),
                'ParentId': parent_id,
                'Score': 0,
                'Id': last_id + 1,
            }
            answer_result = collection_answers.insert_one(new_answer)
            answer_id = answer_result.inserted_id
            collection_questions.update_one({'_id': parent_object_id}, {'$push': {'answers': last_id+1}})
            print(f"Inserted new answer for question ID: {parent_object_id}, Answer ID: {answer_id}")
        count = collection_questions.count_documents(query)
    end_time = time.time()
    mongo_times.append(end_time - start_time)
    print(f"{q['description']}: {count} found in {end_time - start_time:.4f} seconds")

Retrieve title of all questions that have a score greater than 300: 52 found in 0.1680 seconds
Retrieve all answers from 2014: 159322 found in 0.4466 seconds
The deleted tag is: python, the question ID is: 6756c0b4a153ef066a2667b3
Delete tag python for the oldest question from 2015 that has at least 2 tags: 123098 found in 1.1771 seconds
Inserted new answer for question ID: 6756c0b4a153ef066a2836b6, Answer ID: 6756c0d5a153ef066a4769c1
Insert a test answer for the oldest question from 2015 containing the word pdf in the title and that has at least 4 answers: 2 found in 1.1538 seconds


In [ ]:
mysql_times = []
for q in queries:
    start_time = time.time()
    if "score greater" in q["description"]:
        cursor.execute("SELECT title FROM questions WHERE score > 300")
        results = cursor.fetchall()

    elif "answers from 2014" in q["description"]:
        cursor.execute("SELECT * FROM answers WHERE creationdate LIKE '2014%'")
        results = cursor.fetchall()
    
    elif "2 tags" in q["description"]:
        # First, get the question ID using the WITH clause
        cursor.execute("""
            SELECT qu.Id 
            FROM (SELECT q.Id
                FROM questions q
                JOIN taglink t ON q.Id = t.Id
                WHERE q.CreationDate LIKE '2015%'
                GROUP BY q.Id
                HAVING COUNT(t.Tag) >= 2
                ORDER BY MIN(q.CreationDate) DESC) qu
            WHERE 
            EXISTS (SELECT 1 FROM taglink t WHERE t.Id = qu.Id AND t.Tag = 'python')
            LIMIT 1
        """)
        results = cursor.fetchall()
        if results:
            question_id = results[0][0]
            cursor.execute("""
                DELETE FROM taglink 
                WHERE id = %s AND tag = 'python'
            """, (question_id,))
            connection.commit()
            print(f"Tag 'python' deleted from question with id = {question_id}")
    else:
        cursor.execute("""
            SELECT q.Id
            FROM questions q
            JOIN answers a ON q.Id = a.ParentId
            WHERE q.CreationDate LIKE '2015%'
            AND q.Title LIKE '%pdf%'
            AND (SELECT COUNT(*) FROM answers a WHERE a.ParentId = q.Id) >= 4
            ORDER BY q.CreationDate DESC
            LIMIT 1
        """)
        qt_result = cursor.fetchall()
        if qt_result:
            question_id = qt_result[0][0]
            cursor.execute("""
                INSERT INTO answers (OwnerUserId, CreationDate, ParentId, Score, Body)
                VALUES (1, NOW(), %s, 0, 'This is the new answer for this question.')
            """, (question_id,))
            connection.commit()
            print(f"New answer inserted for question with id = {question_id}")
    end_time = time.time()
    mysql_times.append(end_time - start_time)
    print(f"{q['description']}: {len(results)} results found in {end_time - start_time:.4f} seconds")

# Close the cursor and connection
#cursor.close()
#connection.close()

Retrieve title of all questions that have a score greater than 300: 52 results found in 2.6925 seconds
Retrieve all answers from 2014: 159660 results found in 1.5149 seconds
Tag 'python' deleted from question with id = 34552473
Delete tag python for the oldest question from 2015 that has at least 2 tags: 1 results found in 6.7952 seconds
New answer inserted for question with id = 33998802
Insert a test answer for the oldest question from 2015 containing the word pdf in the title and that has at least 4 answers: 1 results found in 2.8130 seconds


In [ ]:
comparison_df = pd.DataFrame({
    "Query": [q["description"] for q in queries],
    "MongoDB_Time(s)": mongo_times,
    "MySQL_Time(s)": mysql_times
})

print(comparison_df)

                                               Query  MongoDB_Time(s)  \
0  Retrieve title of all questions that have a sc...         0.224689   
1                     Retrieve all answers from 2014         0.280378   
2  Delete tag python for the oldest question from...         1.228834   
3  Insert a test answer for the oldest question f...         1.191772   

   MySQL_Time(s)  
0       2.692547  
1       1.514943  
2       6.795220  
3       2.813002  


### Indexes

In [22]:
query = {'Score': {'$gt': 300}}

print("\nQuerying without Index:")
results = list(collection_questions.find(query))
print(f"Results found: {len(results)}")
print("\nExplain output without Index:")
explain_output = collection_questions.find(query).explain()
pprint.pprint(explain_output["executionStats"])


Querying without Index:
Results found: 52

Explain output without Index:
{'allPlansExecution': [],
 'executionStages': {'advanced': 52,
                     'direction': 'forward',
                     'docsExamined': 431737,
                     'executionTimeMillisEstimate': 225,
                     'filter': {'Score': {'$gt': 300}},
                     'isCached': False,
                     'isEOF': 1,
                     'nReturned': 52,
                     'needTime': 431685,
                     'needYield': 0,
                     'restoreState': 11,
                     'saveState': 11,
                     'stage': 'COLLSCAN',
                     'works': 431738},
 'executionSuccess': True,
 'executionTimeMillis': 230,
 'nReturned': 52,
 'totalDocsExamined': 431737,
 'totalKeysExamined': 0}


In [23]:
collection_questions.create_index([("Score",ASCENDING)], name='score')
print("\nIndex on 'Score' created")
print(collection_questions.index_information())


Index on 'Score' created
{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'score': {'v': 2, 'key': [('Score', 1)]}}


In [24]:
print("\nQuerying with Index:")
results = list(collection_questions.find(query))
print(f"Results found: {len(results)}")
print("\nExplain output with Index:")
explain_output = collection_questions.find(query).explain()
pprint.pprint(explain_output["executionStats"])

collection_questions.drop_index("score")


Querying with Index:
Results found: 52

Explain output with Index:
{'allPlansExecution': [],
 'executionStages': {'advanced': 52,
                     'alreadyHasObj': 0,
                     'docsExamined': 52,
                     'executionTimeMillisEstimate': 0,
                     'inputStage': {'advanced': 52,
                                    'direction': 'forward',
                                    'dupsDropped': 0,
                                    'dupsTested': 0,
                                    'executionTimeMillisEstimate': 0,
                                    'indexBounds': {'Score': ['(300, inf.0]']},
                                    'indexName': 'score',
                                    'indexVersion': 2,
                                    'isEOF': 1,
                                    'isMultiKey': False,
                                    'isPartial': False,
                                    'isSparse': False,
                                  

In [25]:
query = {
            'CreationDate': {
                '$gte': datetime(2014, 1, 1),
                '$lte': datetime(2014, 12, 31)
            }
        }

print("\nQuerying without Index:")
results = list(collection_answers.find(query))
print(f"Results found: {len(results)}")
print("\nExplain output without Index:")
explain_output = collection_answers.find(query).explain()
pprint.pprint(explain_output["executionStats"])


Querying without Index:
Results found: 159322

Explain output without Index:
{'allPlansExecution': [],
 'executionStages': {'advanced': 159322,
                     'direction': 'forward',
                     'docsExamined': 679230,
                     'executionTimeMillisEstimate': 397,
                     'filter': {'$and': [{'CreationDate': {'$lte': datetime.datetime(2014, 12, 31, 0, 0)}},
                                         {'CreationDate': {'$gte': datetime.datetime(2014, 1, 1, 0, 0)}}]},
                     'isCached': False,
                     'isEOF': 1,
                     'nReturned': 159322,
                     'needTime': 519908,
                     'needYield': 0,
                     'restoreState': 21,
                     'saveState': 21,
                     'stage': 'COLLSCAN',
                     'works': 679231},
 'executionSuccess': True,
 'executionTimeMillis': 403,
 'nReturned': 159322,
 'totalDocsExamined': 679230,
 'totalKeysExamined': 0}


In [26]:
collection_answers.create_index([("CreationDate",ASCENDING)], name='creationdate')
print("\nIndex on 'CreationDate' created")
print(collection_answers.index_information())


Index on 'CreationDate' created
{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'creationdate': {'v': 2, 'key': [('CreationDate', 1)]}}


In [27]:
print("\nQuerying with Index:")
results = list(collection_answers.find(query))
print(f"Results found: {len(results)}")
print("\nExplain output with Index:")
explain_output = collection_answers.find(query).explain()
pprint.pprint(explain_output["executionStats"])

collection_answers.drop_index("creationdate")


Querying with Index:
Results found: 159322

Explain output with Index:
{'allPlansExecution': [],
 'executionStages': {'advanced': 159322,
                     'alreadyHasObj': 0,
                     'docsExamined': 159322,
                     'executionTimeMillisEstimate': 140,
                     'inputStage': {'advanced': 159322,
                                    'direction': 'forward',
                                    'dupsDropped': 0,
                                    'dupsTested': 0,
                                    'executionTimeMillisEstimate': 50,
                                    'indexBounds': {'CreationDate': ['[new '
                                                                     'Date(1388534400000), '
                                                                     'new '
                                                                     'Date(1419984000000)]']},
                                    'indexName': 'creationdate',
                  

In [28]:
query = {
            'CreationDate': {
                '$gte': datetime(2015, 1, 1),
                '$lte': datetime(2015, 12, 31)
            },
            '$expr': {'$gte': [{'$size': '$tags'}, 2]}
        }

print("\nQuerying without Index:")
results = list(collection_questions.find(query))
print(f"Results found: {len(results)}")
print("\nExplain output without Index:")
explain_output = collection_questions.find(query).explain()
pprint.pprint(explain_output["executionStats"])


Querying without Index:
Results found: 123098

Explain output without Index:
{'allPlansExecution': [],
 'executionStages': {'advanced': 123098,
                     'direction': 'forward',
                     'docsExamined': 431737,
                     'executionTimeMillisEstimate': 341,
                     'filter': {'$and': [{'CreationDate': {'$lte': datetime.datetime(2015, 12, 31, 0, 0)}},
                                         {'CreationDate': {'$gte': datetime.datetime(2015, 1, 1, 0, 0)}},
                                         {'$expr': {'$gte': [{'$size': ['$tags']},
                                                             {'$const': 2}]}}]},
                     'isCached': False,
                     'isEOF': 1,
                     'nReturned': 123098,
                     'needTime': 308639,
                     'needYield': 0,
                     'restoreState': 19,
                     'saveState': 19,
                     'stage': 'COLLSCAN',
                

In [29]:
collection_questions.create_index([("CreationDate",ASCENDING)], name='creationdate')
print("\nIndex on 'CreationDate' created")
print(collection_questions.index_information())


Index on 'CreationDate' created
{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'creationdate': {'v': 2, 'key': [('CreationDate', 1)]}}


In [30]:
print("\nQuerying with Index:")
results = list(collection_questions.find(query))
print(f"Results found: {len(results)}")
print("\nExplain output with Index:")
explain_output = collection_questions.find(query).explain()
pprint.pprint(explain_output["executionStats"])

collection_questions.drop_index("creationdate")


Querying with Index:
Results found: 123098

Explain output with Index:
{'allPlansExecution': [],
 'executionStages': {'advanced': 123098,
                     'alreadyHasObj': 0,
                     'docsExamined': 131899,
                     'executionTimeMillisEstimate': 221,
                     'filter': {'$expr': {'$gte': [{'$size': ['$tags']},
                                                   {'$const': 2}]}},
                     'inputStage': {'advanced': 131899,
                                    'direction': 'forward',
                                    'dupsDropped': 0,
                                    'dupsTested': 0,
                                    'executionTimeMillisEstimate': 31,
                                    'indexBounds': {'CreationDate': ['[new '
                                                                     'Date(1420070400000), '
                                                                     'new '
                                    

In [31]:
query = {'CreationDate': {
                '$gte': datetime(2015, 1, 1),
                '$lte': datetime(2015, 12, 31)
            }, 
            'Title': {'$regex' : '.*pdf.*'}, 
            '$expr': {'$gte': [{'$size': '$answers'}, 4]}
            }

print("\nQuerying without Index:")
results = list(collection_questions.find(query))
print(f"Results found: {len(results)}")
print("\nExplain output without Index:")
explain_output = collection_questions.find(query).explain()
pprint.pprint(explain_output["executionStats"])


Querying without Index:
Results found: 2

Explain output without Index:
{'allPlansExecution': [],
 'executionStages': {'advanced': 2,
                     'direction': 'forward',
                     'docsExamined': 431737,
                     'executionTimeMillisEstimate': 390,
                     'filter': {'$and': [{'CreationDate': {'$lte': datetime.datetime(2015, 12, 31, 0, 0)}},
                                         {'CreationDate': {'$gte': datetime.datetime(2015, 1, 1, 0, 0)}},
                                         {'Title': {'$regex': '.*pdf.*'}},
                                         {'$expr': {'$gte': [{'$size': ['$answers']},
                                                             {'$const': 4}]}}]},
                     'isCached': False,
                     'isEOF': 1,
                     'nReturned': 2,
                     'needTime': 431735,
                     'needYield': 0,
                     'restoreState': 19,
                     'saveState':

In [32]:
collection_questions.create_index([("CreationDate", ASCENDING), ("Title", ASCENDING)], name="creationdate_title")
print("\nCompound Index on 'CreationDate' and 'Title' created")
print(collection_questions.index_information())


Compound Index on 'CreationDate' and 'Title' created
{'_id_': {'v': 2, 'key': [('_id', 1)]}, 'creationdate_title': {'v': 2, 'key': [('CreationDate', 1), ('Title', 1)]}}


In [33]:
print("\nQuerying with Compound Index:")
results = list(collection_questions.find(query))
print(f"Results found: {len(results)}")
print("\nExplain output with Compound Index:")
explain_output = collection_questions.find(query).explain()
pprint.pprint(explain_output["executionStats"])

collection_questions.drop_index("creationdate_title")



Querying with Compound Index:
Results found: 2

Explain output with Compound Index:
{'allPlansExecution': [],
 'executionStages': {'advanced': 2,
                     'alreadyHasObj': 0,
                     'docsExamined': 191,
                     'executionTimeMillisEstimate': 185,
                     'filter': {'$expr': {'$gte': [{'$size': ['$answers']},
                                                   {'$const': 4}]}},
                     'inputStage': {'advanced': 191,
                                    'direction': 'forward',
                                    'dupsDropped': 0,
                                    'dupsTested': 0,
                                    'executionTimeMillisEstimate': 185,
                                    'filter': {'Title': {'$regex': '.*pdf.*'}},
                                    'indexBounds': {'CreationDate': ['[new '
                                                                     'Date(1420070400000), '
                          

In [34]:
# Query without any index (searching by email)
query = "SELECT title FROM questions WHERE score > 300"
print("\nQuerying without index:")
start_time = time.time()
cursor.execute(query)
results = cursor.fetchall()
no_index_time = time.time() - start_time
print(f"Number of results: {len(results)}")
print(f"Query time without index: {no_index_time:.4f} seconds")
cursor.execute(f"EXPLAIN ANALYZE {query}")
explain_result_before = cursor.fetchall()
print("Explain output without index:")
for row in explain_result_before:
    print(row)


# Create index on 'email'
cursor.execute("CREATE INDEX score_index ON questions (score);")
print("\nIndex on 'score' created.")

# Query with the index
print("\nQuerying with index:")
start_time = time.time()
cursor.execute(query)
results = cursor.fetchall()
index_time = time.time() - start_time
print(f"Number of results: {len(results)}")
print(f"Query time with index: {index_time:.4f} seconds")
cursor.execute(f"EXPLAIN ANALYZE {query}")
explain_result_after = cursor.fetchall()
print("Explain output with index:")
for row in explain_result_after:
    print(row)


Querying without index:
Number of results: 52
Query time without index: 1.6038 seconds
Explain output without index:
('-> Filter: (questions.Score > 300)  (cost=79218 rows=131870) (actual time=7.18..1571 rows=52 loops=1)\n    -> Table scan on questions  (cost=79218 rows=395651) (actual time=0.918..1552 rows=431737 loops=1)\n',)

Index on 'score' created.

Querying with index:
Number of results: 52
Query time with index: 0.0111 seconds
Explain output with index:
('-> Index range scan on questions using score_index over (300 < Score), with index condition: (questions.Score > 300)  (cost=59.8 rows=52) (actual time=0.0445..0.242 rows=52 loops=1)\n',)


In [35]:
# Query without any index (searching by email)
query = "SELECT * FROM answers WHERE creationdate LIKE '2014%'"
print("\nQuerying without index:")
start_time = time.time()
cursor.execute(query)
results = cursor.fetchall()
no_index_time = time.time() - start_time
print(f"Number of results: {len(results)}")
print(f"Query time without index: {no_index_time:.4f} seconds")
cursor.execute(f"EXPLAIN ANALYZE {query}")
explain_result_before = cursor.fetchall()
print("Explain output without index:")
for row in explain_result_before:
    print(row)


# Create index on 'email'
cursor.execute("CREATE INDEX creationdate_index ON answers (creationdate);")
print("\nIndex on 'creationdate' created.")

# Query with the index
print("\nQuerying with index:")
start_time = time.time()
cursor.execute(query)
results = cursor.fetchall()
index_time = time.time() - start_time
print(f"Number of results: {len(results)}")
print(f"Query time with index: {index_time:.4f} seconds")
cursor.execute(f"EXPLAIN ANALYZE {query}")
explain_result_after = cursor.fetchall()
print("Explain output with index:")
for row in explain_result_after:
    print(row)


Querying without index:
Number of results: 159660
Query time without index: 1.5315 seconds
Explain output without index:
("-> Filter: (answers.CreationDate like '2014%')  (cost=99242 rows=70030) (actual time=691..1363 rows=159660 loops=1)\n    -> Table scan on answers  (cost=99242 rows=630337) (actual time=0.746..1177 rows=679230 loops=1)\n",)

Index on 'creationdate' created.

Querying with index:
Number of results: 159660
Query time with index: 1.5955 seconds
Explain output with index:
("-> Filter: (answers.CreationDate like '2014%')  (cost=98394 rows=70030) (actual time=725..1429 rows=159660 loops=1)\n    -> Table scan on answers  (cost=98394 rows=630337) (actual time=0.702..1243 rows=679230 loops=1)\n",)
